# Twisted Probabilities, Uncertainty, and Prices
by [Lars Peter Hansen](https://larspeterhansen.org/), [Bálint Szőke](https://www.balintszoke.com/), [Lloyd S. Han](https://sites.google.com/site/lloydhan/) and [Thomas J.Sargent](http://www.tomsargent.com/), you could find the latest draft [here](https://larspeterhansen.org/research/papers/). 

We illustrate effects of concerns about robustness in three environments using a model of capital accumulation with adjustment costs proposed by [Eberly & Wang (2011)](https://www0.gsb.columbia.edu/faculty/nwang/papers/EberlyWang2011_04v3.pdf). We modify their model to expose investment returns  to long-run risks and make investors concerned  about misspecifications of them. Three distinct example economies environments feature:
* a single capital stock
* two capital stocks with a common exposure to long-run risk
*  two capital stocks with only one being exposed to long-run risk

The notebook presents the code for two capitals model accompanying section 6 in the paper

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Baseline-model" data-toc-modified-id="Baseline-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Baseline model</a></span><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Calibration" data-toc-modified-id="Calibration-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Calibration</a></span></li></ul></li><li><span><a href="#Two-Capitals-Stock-Case" data-toc-modified-id="Two-Capitals-Stock-Case-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Two Capitals Stock Case</a></span><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Planner's-problem" data-toc-modified-id="Planner's-problem-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Planner's problem</a></span></li></ul></li><li><span><a href="#Code" data-toc-modified-id="Code-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><span><a href="#Choosing-parameters" data-toc-modified-id="Choosing-parameters-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Choosing parameters</a></span></li><li><span><a href="#Calibration" data-toc-modified-id="Calibration-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Calibration</a></span></li><li><span><a href="#Calculate-Impulse-Response-Functinos" data-toc-modified-id="Calculate-Impulse-Response-Functinos-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Calculate Impulse Response Functinos</a></span></li><li><span><a href="#Save-results" data-toc-modified-id="Save-results-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Save results</a></span></li></ul></li></ul></div>

In [1]:
# Install and import Necessary packages
using Pkg
Pkg.add("Optim")
Pkg.add("Roots")
Pkg.add("NPZ")
Pkg.add("Interpolations")
Pkg.add("Distributed")

using Optim
using Roots
using NPZ
using Distributed
using Interpolations

include("./newsets_utils_d.jl")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


shock_price (generic function with 1 method)

## Baseline model

### Setup

<p style='text-align: justify;'>Aggregate output is proportional to a single capital stock with a constant productivity parameter $\mathcal{A}$. A representative household cares about consumption $C$ with instantaneous utility $\delta\log C$. Under the baseline model, investment $I$ affects capital $K$ according to</p>

\begin{align}
dK_t &= \left[\frac{I_t}{K_t} - \frac{\phi}{2}\left(\frac{I_t}{K_t}\right)^2 + (.01)\left(\widehat \alpha_k + \widehat \beta Z_t \right)\right]K_t dt + (.01)\sigma_k K_t\cdot dW_t \label{eqn:model}\\
dZ_t &= \left(\widehat\alpha_z -\widehat \kappa Z_t \right)dt + \sigma_z \cdot dW_t \nonumber
\tag{1}
\end{align}

<p style='text-align: justify;'>with adjustment cost parameter $\phi$. With zero investment, the rate of change of capital is $(.01)\left(\widehat\alpha_k + \widehat\beta Z_t\right)$, where $Z$ is an exogenously specified continuous-time scalar autoregressive process  that puts long-term risks into  returns on capital. Under baseline model $(1)$, the mean of $Z$ is  ${\overline z} = {\frac {{\widehat \alpha}_z} {{\widehat \kappa}}}$.</p>

<p style='text-align: justify;'>he aggregate resource constraint is $C + I = {\cal A} K$. It is convenient to pose a robust planner's problem in terms of an investment-capital ratio $D_t \doteq\frac{I_t}{K_t}$ and $\log K_t$ that satisfy</p>

\begin{align*}
d \log K_t = \left[D_t - \frac{\phi}{2}\left(D_t\right)^2 + (.01)\left(\widehat \alpha_k + \widehat \beta Z_t \right) - {\frac {(.01)^2 |\sigma_k|^2}{2}} \right] dt + (.01)\sigma_k \cdot dW_t.
\end{align*}

Denote state variable as $X_t = [\log K_t,\ Z_t]'$ and instantaneous utility function

\begin{align*}
\upsilon(X_t, D_t) = \delta\log\left({\mathcal A}- D_t\right) + \delta\log K_t.
\end{align*}



### Calibration

<p style='text-align: justify;'>The basis of our quantitative analysis is an empirical model of  aggregate consumption dynamics. Our code for VAR estimation could be found in <em><b>VAR Estimation</b></em> section of Git hub repository <a href="https://github.com/lphansen/TenuousBeliefs">Tenuous Belief</a>.</p>

<p style='text-align: justify;'>We follow <a href = "http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf">Hansen et al. (2008) </a> by fitting a trivariate VAR for macroeconomic time series that contain information about long-term consumption growth, namely,
log consumption growth, the difference between logs of business income and consumption, and the difference between logs of personal dividend income and
consumption.</p>

<p style='text-align: justify;'>The time series are quarterly data from 1948 Q1 to 2018 Q2. Our consumption measure is per capita consumption of non-durables and services from NIPA Table 1.1.5. Business income is measured as proprietor's income plus corporate profits per capita and the series are from NIPA Table 1.12. Personal dividend income is from NIPA Table 2.1. By including proprietors' income in addition to corporate profits, we use a broader measure of business income than <a href = "http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf">Hansen et al. (2008) </a> , who used only corporate profits. Moreover, <a href = "http://larspeterhansen.org/wp-content/uploads/2016/10/Consumption-Strikes-Back-Measuring-Long-Run-Risk.pdf">Hansen et al. (2008) </a>  did not include  personal dividends in their VAR analysis.
We restrict  all three time series to have a common martingale component by imposing a known cointegration relation among them.</p>

<p style='text-align: justify;'>We convert the discrete time VAR estimates to baseline parameters $({\widehat \alpha}_k,  {\widehat \beta}, {\widehat \alpha}_z, {\widehat \kappa} )$ and $(\sigma_c, \sigma_z)$ by setting ${\widehat \alpha}_z=0$ and $\widehat{\beta}=1$ and matching the dynamics of the VAR implied long-term consumption growth forecast with those of $Z$.
As a result, we obtain the following parameters for the baseline model of consumption $(1)$:</p>

\begin{align} 
& \begin{matrix}
{\widehat \alpha}_c  & = & .484 & & {\widehat \beta} &= &1 \cr
{\widehat \alpha}_z  &= & 0 &  & {\widehat \kappa} & = & .014 \end{matrix} \cr
& \sigma = \begin{bmatrix}
(\sigma_c)' \cr (\sigma_z)' \end{bmatrix}  =  \begin{bmatrix} .477 & 0 \cr  .011 & .025 \end{bmatrix} .
\label{estimations}
\end{align}

We set the household's subjective discount rate equal to $\delta=.002.$

<p style='text-align: justify;'>In more detail, we choose $\widehat{\alpha}_c$ and $(1-\widehat{\kappa})$ to match the VAR implied unconditional mean of consumption growth and autoregressive coefficient of expected long-term consumption growth, respectively. In addition, we set $(\sigma_c, \sigma_z)$ such that $(1, \widehat{\kappa}^{-1})\sigma\sigma'(1, \widehat{\kappa}^{-1})'$ is equal to the VAR implied covariance matrix of the one-step-ahead forecast error for consumption growth and expected long-term consumption growth. We achieve identification by imposing a lower triangular structure with positive diagonal elements on $\sigma$.</p>

In [2]:
#==============================================================================#
#    PARAMETERS
#==============================================================================#
delta = 0.002;

# (1) Baseline model
alpha_z_hat = 0.0;
kappa_hat = 0.014;

alpha_c_hat = 0.484;       # consumption intercept (estimated) 
beta_hat = 1.0;
sigma_c = [0.477, 0.0  ];   # consumption exposure (= exposure of single capital)
sigma_z = [0.011, 0.025];

## Two Capitals Stock Case

### Setup
<p style='text-align: justify;'>We now extend investment opportunities by adding a second productive capital that can be used to produce the common consumption good with constant productivity. Under the baseline model, investment $I^{(1)}$ and $I^{(2)}$ affect two capital stocks $K^{(1)}$ and $K^{(2)}$ according to</p>

\begin{align*}
dK_t^{(1)} &= K_t^{(1)} \left[\left({\frac {I_t^{(1)}}{K_t^{(1)}}}  - {\frac {\phi_1}  2}  \left(\frac{I_t^{(1)}}{K_t^{(1)}} \right)^2 + (.01) \left( {\widehat \alpha}_1 + {\widehat \beta}_1 Z_t   \right)\right) dt + (.01) \sigma_1   \cdot dW_t \right] \\
dK_t^{(2)} &=  K_t^{(2)} \left[ \left({\frac{I_t^{(2)}}{K_t^{(2)}}}  -   {\frac {\phi_2}  2} \left(\frac{I_t^{(2)}}{K_t^{(2)}} \right)^2 + (.01) \left( {\widehat \alpha}_2 + {\widehat \beta}_2 Z_t \right) \right)  dt + (.01)  \sigma_{2}  \cdot dW_t \right] \\
dZ_t &= \left({\widehat \alpha}_z   - {\widehat \kappa} Z_t \right)dt + \sigma_z \cdot dW_t
\tag{2}
\end{align*}
subject to the aggregate resource constraint

\begin{align*}
C_t + I_t^{(1)} + I_t^{(2)} = \mathcal{A}_1  K_t^{(1)} + \mathcal{A}_2 K_t^{(2)}.
\end{align*}

<p style='text-align: justify;'>The two sectors are  identical in their technology parameters $\left(\mathcal{A}_1, \phi_1, \widehat{\alpha}_1\right)=\left(\mathcal{A}_2, \phi_2, \widehat{\alpha}_2\right)$ and exposures to the Brownian shocks, $\sigma_1=\sigma_2$, but  can differ in  exposures to long-run risk, so that $\widehat{\beta}_1 \neq \widehat{\beta}_2$. To study how multiple capital stocks and  heterogeneity in their exposures to  long-run risk  affect decision rules and the worst-case model, we consider two cases:</p>
<ul>
<li> <em><b>ex post heterogeneity:</b></em> two capital stocks possess identical evolution equations but are exposed to idiosyncratic shocks that give rise to a non degenerate distribution of capital ex post. This case features a trade-off between diversification and adjustment costs similar to studied by <a ref = 'https://www0.gsb.columbia.edu/faculty/nwang/papers/EberlyWang2011_04v3.pdf'> Eberly & Wang (2011)</a>. We study how concerns about model misspecification affect this trade-off.</li>

<li> <em><b>ex ante heterogeneity:</b></em> two capital stocks differ in their evolution equations, the first capital stock is immune to long-run risk because $\widehat{\beta}_1=0$, while the second capital stock is exposed to it because  $\widehat{\beta_2}>0$. %and alter the technology parameters so that the induced average consumption dynamics is identical to the single-capital economy.</li>
</ul>

<p style='text-align: justify;'>Like the case with a single capital stock, it is convenient to use the investment ratios $D_t^{(1)} \doteq {\frac {I_t^{(1)}}{K_t^{(1)}}}$ and $ D_t^{(2)} \doteq {\frac {I_t^{(2)}}{K_t^{(2)}}}$ as controls, and the logarithm of aggregate capital, $\log K_t \doteq \log\left(K^{(1)}+K^{(2)}\right)$, and the long-run risk component, $Z$, as state variables. However, with  costly reallocation between the two capital stocks, the distribution of capital becomes an additional endogenous state variable that we express with the ratio</p>

\begin{align*}
R_t \doteq \frac{K^{(2)}_t}{K^{(1)}_t+ K^{(2)}_t} = \frac{K^{(2)}_t}{K_t} \in [0, 1].
\end{align*}

<p style='text-align: justify;'>Adjustment costs prevent the household from setting  $R_t$ ideally at every instant; instead the household  influences its motion at each instant  by setting the investment ratios $D_t^{(1)}$ and $D_t^{(2)}$.</p>

The state vector is then $X_t \doteq \left[\log K_t, \ R_t, \ Z_t\right]'$. 

### Planner's problem

<p style='text-align: justify;'>In Appendix C, we show that the value function of the robust planner is additively separable in $\log K$, making the optimal investment ratios $d_1^*$ and $d_2^*$ become functions of $(R, Z)$ only. Implied equilibrium consumption is</p>
    
\begin{align}
\log C_t = \log K_t + \log\Big([\mathcal{A}_1 - d^*_1(R_t, Z_t)](1-R_t) + [\mathcal{A}_2-d^*_2(R_t, Z_t)]R_t \Big).
\end{align}
<p style='text-align: justify;'>The worst-case model ceases to share the  parametric form of  the baseline model, but the distortion $h^*$ remains Markov and depends only on the contemporaneous capital distribution, $R_t$, and the long-run risk state $Z_t$.</p>

<p style='text-align: justify;'>To induce independent variation in the two capital return processes, we presume that there are three underlying Brownian motions with volatility vectors</p>

\begin{align*}
\sigma_1 &= \mathsf{s}\begin{bmatrix}
.477  \\
0 \\
0
\end{bmatrix},\quad \sigma_2 = \mathsf{s}\begin{bmatrix}
0 \\
.477 \\
0
\end{bmatrix}, \quad \sigma_z = \begin{bmatrix}
.011\sqrt{.5} \\
.011 \sqrt{.5}\\
.025
\end{bmatrix} ,
\end{align*}

<p style='text-align: justify;'>where $\mathsf{s}$ is a scaling parameter that we introduce to control  effects of diversification on aggregate consumption volatility. Multiplying the first two entries of $\sigma_z$ by $\sqrt{.5}$ ensures that, despite having an extra shock, the local volatility of the long-run risk state, $|\sigma_z|$, is unchanged relative to the single capital economy.</p>

The state vector is
\begin{align*}
X_t \doteq \left[\log K_t, \ L_t, \ Z_t-\bar{z}\right]' \quad\quad \log K_t \doteq \log\left(K^{(1)}_t+ K^{(2)}_t\right)\quad\quad L_t \doteq \log K^{(2)}_t - \log K^{(1)}_t
\end{align*}

<p style='text-align: justify;'>For numerical stability concerns, we used $L_t$ defined above instead of capital distribution ratio $R_t$. One can get $R_t$ from $L_t$ using the one-to-one mapping $R_t=\exp(L_t)/(1+\exp(L_t))$. </p>

The period utility function is
\begin{align*}
\upsilon(X, D) &= \delta \log\left((1-R)\left(\mathcal{A}_1 - D^{(1)}\right) + R\left(\mathcal{A}_2 - D^{(2)}\right) \right) + \delta \log K
\end{align*}

<p style='text-align: justify;'>Denote  expected capital growth $E_t\left[dK^{(i)}_t/K^{(i)}_t\right]$ for $i=1, 2$ as
$$\varphi_i\left(D_t^{(i)}, Z_t\right) \doteq D_t^{(i)} - \frac{\phi_i}{2}\left(D_t^{(i)}\right)^2 + (.01)\left(\widehat\alpha_z + \widehat\beta Z_t\right)$$
State variables then follow</p>

\begin{align*}
d\log K_t &= \left[\varphi_1 (1-R_t) + \varphi_2 R_t - \frac{(.01)^2\left|\sigma_1(1-R_t) + \sigma_2 R_t\right|^2}{2}\right]dt + %\\
%& \hspace{1cm} +
(.01)\left[\sigma_1(1-R_t) + \sigma_2 R_t\right]\cdot dW_t \\
dL_t &= \left[\varphi_2 - \varphi_1 -  \frac{(.01)^2}{2}\left(|\sigma_2|^2 - |\sigma_1|^2 \right)\right] dt + (.01)\left[\sigma_2 -\sigma_1\right]\cdot dW_t \\
dZ_t &= - \widehat\kappa \left(Z_t - \bar z\right)dt + \sigma_z \cdot dW_t
\end{align*}

Using Ito's lemma, we can derive the following  dynamics for $R_t$:

\begin{align*}
dR_t &= R_t(1-R_t)\left[\varphi_2 - \varphi_1 +
(.01)^2\left(|\sigma_1|^2(1-R_t) - |\sigma_2|^2 R_t + \sigma'_1\sigma_2(2R_t-1)\right)\right] dt + 
R_t(1-R_t)(.01)\left[\sigma_2 -\sigma_1\right]\cdot dW_t .
\end{align*}

Let $\sigma$ denote the stacked volatility matrix

\begin{align*}
\sigma(X_t) \doteq \begin{bmatrix}
(.01)\left(\sigma'_1(1-R_t) + \sigma'_2 R_t\right) \\
(.01)[\sigma_2-\sigma_1]' \\
\sigma'_z
\end{bmatrix} .
\end{align*}

We seek a value function  $V(X) = \log K + \nu(L, Z)$ that solves the HJB equation

\begin{align}
0 &= \max_{d^{(1)}, d^{(2)}}\min_{h} \ \delta\log\left((1-r)\left(\mathcal A_1 - d^{(1)}\right) + r\left(\mathcal A_2 - d^{(2)}\right) \right) - \delta\nu(l, z) + \frac{\ell}{2}\left[|h|^2 - \xi(z)\right] \nonumber \\
& \quad\quad + \left[\varphi_1(1-r) + \varphi_2r - \frac{(.01)^2\left[\sigma_1(1-r) + \sigma_2 r\right]^2}{2} + (.01)[(1-r)\sigma_1 + r\sigma_2]\cdot h\right]  \nonumber \\
& \quad\quad + \nu_l(l, z)\left[ \varphi_2 - \varphi_1 - \frac{(.01)^2}{2}\left(|\sigma_2|^2  - |\sigma_1|^2 \right) + (.01)[\sigma_2 - \sigma_1]\cdot h\right] \nonumber \\
& \quad\quad + \nu_z(l, z)\left[ -\widehat{\kappa}(z-\bar z) + \sigma_z \cdot h\right] + \frac{1}{2}\text{tr}\left(V_{xx}\sigma\sigma'\right)\label{eq:HJB}
\end{align}

where
\begin{align*}
\text{tr}\left(V_{xx}\sigma\sigma'\right) &= (.01)^2|\sigma_2-\sigma_1|^2 \nu_{ll}(l, z) +  2 (.01)\left([\sigma_2 - \sigma_1]\cdot \sigma_z\right)\nu_{lz}(l, z) +  |\sigma_z|^2 \nu_{zz}(l, z).
\end{align*}

We assume that a  Bellman-Isaacs condition holds so that first-order conditions can be stacked

\begin{align}
\frac{\delta(1-r)}{(1-r)(\mathcal A_1 - d^{(1)}(l, z))+r(\mathcal A_2 - d^{(2)}(l, z))} &= \left(1-\phi_1d^{(1)}(l, z)\right)\left[1 - r - \nu_l(l, z)\right] \label{eq:d1_opt} \\
\frac{\delta r}{(1-r)(\mathcal A_1 - d^{(1)}(l, z))+r(\mathcal A_2 - d^{(2)}(l, z))} &= \left(1-\phi_2d^{(2)}(l, z)\right)\left[r + \nu_l(l, z)\right] \label{eq:d2_opt} \\
h(l, z, \ell^*) &= - \frac{1}{\ell^*} \sigma'(r)\begin{bmatrix}
1 \\
\nu_l(l, z) \\
\nu_z(l, z)
\end{bmatrix} . \label{eq:h}
\end{align}

<p style='text-align: justify;'>These equations determine  optimal investment-capital ratios $d^{(1)}(l, z)$ and $d^{(2)}(l, z)$, and also the worst-case drift distortion $h(l, z)$.
Here  $\ell^*$ is the multiplier that makes the minimizing agent's constraint bind for a given initial $(l_0, z_0)$:</p>

\begin{align*}
\ell^*(l_0, z_0) = \text{arg}\max_\ell \ \ \nu(l_0, z_0, \ell).
\end{align*}

For codes solving planner's problem in single stock capital model, please refer to __newsets_twocapitals.jl__ under __code__ folder

## Code 

### Choosing parameters
Specify following four parameters for running different cases in the paper. User may also change $\tilde{\kappa}$ and $\hat{\beta_2}$ for using different twisting functions. 

In [7]:
symmetric_returns    = 1     # 0 for symmetric return case, 1 for asymmetric returns
state_dependent_xi   = 0     # 0 for constant twisting function, 1 for twisting beta_tilde, 2 for twsting kappa_tilde; 1 and 2 are state dependent cases
optimize_over_ell    = 0     # whether estimating lagrange multiplier 
compute_irfs         = 0     # 1 if one wants to compute irfs, it's fine grid and wold be much more slower
gamma = 8

if compute_irfs == 1
    @everywhere include("./newsets_utils_d.jl")
elseif compute_irfs ==0
    include("./newsets_utils_d.jl")
end

println("=============================================================")
if symmetric_returns == 1
    println(" Economy with two capital stocks: SYMMETRIC RETURNS          ")
    if state_dependent_xi == 0
        println(" No tilting (xi is NOT state dependent)                      ")
        filename = (compute_irfs==0) ? "model_sym_HS.npz" : "model_sym_HS_p.npz";
    elseif state_dependent_xi == 1
        println(" With tilting (change in kappa)                        ")
        filename = (compute_irfs==0) ? "model_sym_HSHS.npz" : "model_sym_HSHS_p.npz";
    elseif state_dependent_xi == 2
        println(" With tilting (change in beta)                        ")
        filename = (compute_irfs==0) ? "model_sym_HSHS2.npz" : "model_sym_HSHS2_p.npz";
    end
elseif symmetric_returns == 0
    println(" Economy with two capital stocks: ASYMMETRIC RETURNS         ")
    if state_dependent_xi == 0
        println(" No tilting (xi is NOT state dependent)                      ")
        filename = (compute_irfs==0) ? "model_asym_HS.npz" : "model_asym_HS_p.npz";
    elseif state_dependent_xi == 1
        println(" With tilting (change in kappa)                        ")
        filename = (compute_irfs==0) ? "model_asym_HSHS.npz" : "model_asym_HSHS_p.npz";
    elseif state_dependent_xi == 2
        println(" With tilting (change in beta)                        ")
        filename = (compute_irfs==0) ? "model_asym_HSHS2.npz" : "model_asym_HSHS2_p.npz";
    end
end


#===========================  CALIBRATION  ====================================#
# consumption_investment = 3.1;
#A_1cap, phi_1cap, alpha_k_hat, investment_capital = calibration2(15.0,
#                                             consumption_investment,
#                                             alpha_c_hat, delta, sigma_c)
# A_1cap, phi_1cap, alpha_k_hat = calibration3(investment_capital,
#                                   consumption_investment,
#                                   alpha_c_hat, delta, sigma_c)
#

A_1cap = 0.05
phi_1cap = 28.0
investment_capital, consumption_investment, alpha_k_hat = calibration3(phi_1cap,
                                            A_1cap, delta, alpha_c_hat, sigma_c)

println("  Calibrated values: A:", A_1cap,
        "  phi_1cap: ", phi_1cap,
        "  alpha_k : ", alpha_k_hat,
        "  C/I : ", consumption_investment,
        "  I/K : ", investment_capital)
println("=============================================================")
#==============================================================================#

# (1) Baseline model
alpha_z_hat = 0.0;
kappa_hat = 0.014;
zbar = alpha_z_hat/kappa_hat;
sigma_z_1cap = [0.011, 0.025];

sigma_z =  [0.011*sqrt(0.5)   , 0.011*sqrt(0.5)   , 0.025];


if symmetric_returns == 1

    beta2_hat = beta1_hat = 0.5;

    # (2) Technology
    phi2 = phi1 = phi_1cap;
    A2 = A1 = A_1cap;

    if state_dependent_xi == 0
        # Constant tilting function
        scale = 1.754;
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat;

        # Worrisome model
        alpha_z_tilde  = -0.005
        kappa_tilde    = kappa_hat;
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat

        # ell_star = 0.055594409575544096
        ell_star = 1/(gamma-1)

    elseif state_dependent_xi == 1
        # State-dependent tilting function (fixed kappa, alpha targets q)
        scale = 1.62
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat;

        alpha_z_tilde  = -0.00155;
        kappa_tilde    =  0.005
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat

        ell_star = 0.13852940062708508

    elseif state_dependent_xi == 2
        # State-dependent tilting function
        scale = 1.568
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat;

        alpha_z_tilde  = -0.00155;
        kappa_tilde    = kappa_hat
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat + .1941
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat + .1941

        ell_star = 0.18756641482672026

    end


elseif symmetric_returns == 0

    beta1_hat = 0.0;
    beta2_hat = 0.5;

    # (2) Technology
    phi2 = phi1 = phi_1cap;
    A2 = A1 = A_1cap;

    if state_dependent_xi == 0
        # Constant tilting function
        scale = 1.307
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat;

        # Worrisome model
        alpha_z_tilde  = -0.00534;
        kappa_tilde    = kappa_hat;
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat

        ell_star = 0.026320287107624605

    elseif state_dependent_xi == 1
        # State-dependent tilting function (fixed kappa, alpha targets q)
        scale = 1.14
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat + 0.035; #.034;

        alpha_z_tilde  = -0.002325
        kappa_tilde    = 0.005;
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat;
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat

        ell_star = 0.04226404306515605

    elseif state_dependent_xi == 2
        # State-dependent tilting function (fixed beta1, alpha targets q)
        scale = 1.27
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat

        alpha_z_tilde  = -0.002325
        kappa_tilde    = kappa_hat
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat + 0.194 #.195
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat + 0.194 #.195

        ell_star = 0.06678494013273199

    end

end

sigma_k1 = [0.477*sqrt(scale),               0.0,   0.0];
sigma_k2 = [0.0              , 0.477*sqrt(scale),   0.0];



# (3) GRID
# For analysis
if compute_irfs == 1
    II, JJ = 7001, 501;     # number of r points, number of z points
    rmax = 4.0;
    rmin = -rmax;
    zmax = 0.7;
    zmin = -zmax;
elseif compute_irfs == 0
    II, JJ = 1001, 201;
    rmax =  18.0;
    rmin = -rmax       #-25.0; #-rmax;
    zmax = 1.0;
    zmin = -zmax;
end

# For the optimization (over ell)
II_opt, JJ_opt = 501, 201;     # number of r points, number of z points
rmax_opt = 18.0;
rmin_opt = -rmax_opt;
zmax_opt = 1.2;
zmin_opt = -zmax_opt;


# (4) Iteration parameters
maxit = 500;        # maximum number of iterations in the HJB loop
crit  = 10e-6;      # criterion HJB loop
Delta = 1000.0;      # delta in HJB algorithm





 Economy with two capital stocks: SYMMETRIC RETURNS          
 No tilting (xi is NOT state dependent)                      
  Calibrated values: A:0.05  phi_1cap: 28.0  alpha_k : -1.2790328319261377  C/I : 0.5727486121839516  I/K : 0.03179147615369309


### Calibration

In [8]:
# Initialize model objects -----------------------------------------------------
baseline = Baseline(alpha_z_hat, kappa_hat, sigma_z_1cap,
                    alpha_c_hat, beta_hat, sigma_c, delta);
baseline1 = Baseline(alpha_z_hat, kappa_hat, sigma_z,
                        alpha_k1_hat, beta1_hat, sigma_k1, delta);
baseline2 = Baseline(alpha_z_hat, kappa_hat, sigma_z,
                        alpha_k2_hat, beta2_hat, sigma_k2, delta);
technology = Technology(A_1cap, phi_1cap);
technology1 = Technology(A1, phi1);
technology2 = Technology(A2, phi2);
model = TwoCapitalEconomy(baseline1, baseline2, technology1, technology2);

worrisome = TwoCapitalWorrisome(alpha_z_tilde, kappa_tilde,
                                alpha_k1_tilde, beta1_tilde,
                                alpha_k2_tilde, beta2_tilde);
worrisome_noR = TwoCapitalWorrisome(alpha_z_hat, kappa_hat,
                                    alpha_k1_hat, beta1_hat,
                                    alpha_k2_hat, beta2_hat);

grid = Grid_rz(rmin, rmax, II, zmin, zmax, JJ);
grid_opt = Grid_rz(rmin_opt, rmax_opt, II_opt, zmin_opt, zmax_opt, JJ_opt);
params = FinDiffMethod(maxit, crit, Delta);

xi0, xi1, xi2 = tilting_function(worrisome, model);


#==============================================================================#
# WITHOUT ROBUSTNESS (indicated by _noR endings)
#==============================================================================#
# println(" (1) Compute value function WITHOUT ROBUSTNESS")

# @time A_noR, V_noR, val_noR, d1_F_noR, d2_F_noR, d1_B_noR, d2_B_noR,
#         h1_F_noR, h2_F_noR, hz_F_noR, h1_B_noR, h2_B_noR, hz_B_noR,
#         mu_1_F_noR, mu_1_B_noR, mu_r_F_noR, mu_r_B_noR, mu_z_noR,
#         V0, rr, zz, pii, dr, dz = value_function_twocapitals(Inf, model,
#                                                             worrisome_noR,
#                                                             grid, params,
#                                                             symmetric_returns);

# g_noR_dist, g_noR = stationary_distribution(A_noR, grid)
# mu_1_noR = (mu_1_F_noR + mu_1_B_noR)/2.0;
# mu_r_noR = (mu_r_F_noR + mu_r_B_noR)/2.0;
println("=============================================================")


if symmetric_returns == 0
    if state_dependent_xi == 0
        params.Delta = 14.0;
    elseif state_dependent_xi == 1
        params.Delta = 17.0;
    elseif state_dependent_xi == 2
        params.Delta = 9.5
    end
end

#==============================================================================#
# WITH ROBUSTNESS
#==============================================================================#
if optimize_over_ell == 1
    println(" (2) ell_star is computed from optimization:                   ")
    #---------------------------------------------------------------------------
    # INITIAL GUESS
    #   when r=-inf/inf, we've single capital case, so we know the value funcs
    #---------------------------------------------------------------------------
    w1_single = Worrisome(worrisome.alpha_z_tilde, worrisome.kappa_tilde,
                            worrisome.alpha_k1_tilde, worrisome.beta1_tilde)
    w2_single = Worrisome(worrisome.alpha_z_tilde, worrisome.kappa_tilde,
                            worrisome.alpha_k2_tilde, worrisome.beta2_tilde)

    res1 = optimize(r-> -value_function(r, zbar, w1_single,
                                        baseline1, technology1)[1], 0.0, 100);
    res2 = optimize(r-> -value_function(r, zbar, w2_single,
                                        baseline2, technology2)[1], 0.0, 100);
    ell = (Optim.minimizer(res1) + Optim.minimizer(res2))/2;


    if (symmetric_returns == 1) && (state_dependent_xi == 2)
        ell_res = optimize(ee -> -value_function_twocapitals(ee, model, worrisome,
                                                                grid_opt, params,
                                                                symmetric_returns)[3],
                            ell*0.7, ell*1.3);
        ell_star = Optim.minimizer(ell_res);
        println("  min ell: ", ell*0.7,"  ell_star: ", ell_star,"  init ell: ", ell*1.3);
    else
        ell_res = optimize(ee -> -value_function_twocapitals(ee, model, worrisome,
                                                                grid_opt, params,
                                                                symmetric_returns)[3],
                            ell*0.5, ell);
        ell_star = Optim.minimizer(ell_res);
        println("  min ell: ", ell*0.5,"  ell_star: ", ell_star,"  init ell: ", ell);
    end

elseif optimize_over_ell == 0

    println(" (2) ell_star is given, it is ", ell_star)

end
println("=============================================================")


println(" (3) Compute value function WITH ROBUSTNESS")
A, V, val, d1_F, d2_F, d1_B, d2_B, h1_F, h2_F, hz_F, h1_B, h2_B, hz_B,
        mu_1_F, mu_1_B, mu_r_F, mu_r_B, mu_z, V0, rr, zz, pii, dr, dz =
        value_function_twocapitals(ell_star, model, worrisome,
                                    grid, params, symmetric_returns);
one_pii = 1.0 .- pii

 (2) ell_star is given, it is 0.14285714285714285
 (3) Compute value function WITH ROBUSTNESS
----------------------------------
Iteration = 1
Distance = 4.680922921361379
v max = -7.611789597082103
v min = -8.246492825284681
d1 max = 0.0017142857142857144
d1 min = 0.0017142857142857144
d2 max = 0.0017142857142857144
d2 min = 0.0017142857142857144
h1 max = 0.008748003786960529
h1 min = 0.002430679656541339
h2 max = 0.008748003786960529
h2 min = 0.002430679656541339
hz max = 0.007812500000000666
hz min = 0.007812499999999556
----------------------------------
----------------------------------
Iteration = 2
Distance = 1.0767435837796517
v max = -8.678760784780156
v min = -9.322168525838315
d1 max = 0.003347642064240194
d1 min = 0.0033393448957908236
d2 max = 0.0033476424963494213
d2 min = 0.003339344895790788
h1 max = 0.008748003719549917
h1 min = 0.0006672084873709641
h2 max = 0.008748003719547492
h2 min = 0.000667208487362236
hz max = 0.007812499793269367
hz min = 0.002144488980198034

1001×201 Matrix{Float64}:
 1.0         1.0         1.0         …  1.0         1.0         1.0
 1.0         1.0         1.0            1.0         1.0         1.0
 1.0         1.0         1.0            1.0         1.0         1.0
 1.0         1.0         1.0            1.0         1.0         1.0
 1.0         1.0         1.0            1.0         1.0         1.0
 1.0         1.0         1.0         …  1.0         1.0         1.0
 1.0         1.0         1.0            1.0         1.0         1.0
 1.0         1.0         1.0            1.0         1.0         1.0
 1.0         1.0         1.0            1.0         1.0         1.0
 1.0         1.0         1.0            1.0         1.0         1.0
 1.0         1.0         1.0         …  1.0         1.0         1.0
 1.0         1.0         1.0            1.0         1.0         1.0
 1.0         1.0         1.0            1.0         1.0         1.0
 ⋮                                   ⋱                          ⋮
 2.26298e-8  2.26298e-8 

In [ ]:

println("=============================================================")

# Define Policies object
policies  = PolicyFunctions(d1_F, d2_F, d1_B, d2_B,
                            -h1_F/ell_star, -h2_F/ell_star, -hz_F/ell_star,
                            -h1_B/ell_star, -h2_B/ell_star, -hz_B/ell_star);
policies2 = PolicyFunctions(d1_F_noR, d2_F_noR, d1_B_noR, d2_B_noR,
                            -h1_F/ell_star, -h2_F/ell_star, -hz_F/ell_star,
                            -h1_B/ell_star, -h2_B/ell_star, -hz_B/ell_star);

# Construct drift terms under the baseline
mu_1 = (mu_1_F + mu_1_B)/2.0;
mu_r = (mu_r_F + mu_r_B)/2.0;
# ... under the worst-case model
h1_dist = (policies.h1_F + policies.h1_B)/2.0;
h2_dist = (policies.h2_F + policies.h2_B)/2.0;
hz_dist = (policies.hz_F + policies.hz_B)/2.0;

WCDist_1, WCDist_r, WCDist_z = worstcase_distortion(h1_dist, h2_dist, hz_dist,
                                                    pii, model);
mu_1_wc = mu_1 + WCDist_1;
mu_r_wc = mu_r + WCDist_r;
mu_z_wc = mu_z + WCDist_z;


# Kolmogorov Forward equation under the baseline model
g_dist, g = stationary_distribution(A, grid)

# Kolmogorov Forward equation under the worst-case model
A_wc = Kolmogorov_FinDiff(policies, model, grid, params);
g_wc_dist, g_wc = stationary_distribution(A_wc, grid);

# Kolmogorov Forward eq under the worst-case using non-robust decision rule
A_wc_noR = Kolmogorov_FinDiff(policies2, model, grid, params);
g_wc_noR_dist, g_wc_noR = stationary_distribution(A_wc_noR, grid);

#==============================================================================#
# Approximate relative entropy (need worst-case distribution)
#==============================================================================#
println(" (4) Compute distance between worst-case and baseline")
# local uncertainty prices
h1, h2, hz = -h1_dist, -h2_dist, -hz_dist;

H2 = h1.^2 + h2.^2 + hz.^2;
re = sum(H2 .* g_wc * dz*dr)/2;
q = sqrt(re*2);

# CHERNOFF ENTROPY
#gamma_res = optimize(gamma -> chernoff_objective(gamma, policies, model,
#                                                 grid, params), 1e-5, 1-1e-5);
#chernoff = - Optim.minimum(gamma_res);
#halflife = log(2) / chernoff;

chernoff = 0.0;
halflife = 0.0;

println("    alpha_tilde: ", worrisome.alpha_z_tilde,
        "  kappa_tilde: ", worrisome.kappa_tilde,
        "  q: ", q,
        "  re: ", re,
        "  chernoff: ", chernoff,
        "  halflife: ", halflife);
println("=============================================================")


#==============================================================================#
# Stationary distributions of local uncertainty prices
#==============================================================================#
inner = 1
inI = (inner+1):(II-inner)
inJ = (inner+1):(JJ-inner)

# Single capital
H0, H1 = 0.0, 0.0
# H0, H1 = worst_case(alpha_z_tilde, kappa_tilde, alpha_k2_tilde, beta2_tilde,
#                     baseline, technology)[1:2]
stdev_z_1cap = sqrt(dot(sigma_z_1cap, sigma_z_1cap)/(2*kappa_hat));

h12_vec, h12_density = change_of_variables((h1+h2)/sqrt(2),g,rr,zz, inner)
hz_vec, hz_density   = change_of_variables(hz, g, rr, zz, inner)

# Two capitals economy
#if symmetric_returns == 1
#h12_vec, h12_density = change_of_variables((h1+h2)/sqrt(2),g,rr,zz,inner)
#elseif symmetric_returns == 0
#    h12_vec, h12_density = change_of_variables(h2, g, rr, zz, inner)
#end

#==============================================================================#
# Stationary distributions of consumption
#==============================================================================#
cons_1cap = technology.A - dstar_singlecapital(technology, baseline)
d1_noR = (d1_F_noR + d1_B_noR)/2;
d2_noR = (d2_F_noR + d2_B_noR)/2;
d1 = (policies.d1_F + policies.d1_B)/2;
d2 = (policies.d2_F + policies.d2_B)/2;

cons_noR = one_pii .* (model.t1.A .- d1_noR) + pii .* (model.t2.A .- d2_noR)
cons     = one_pii .* (model.t1.A .- d1) + pii .* (model.t2.A .- d2);

cons_noR_vec, cons_noR_density = change_of_variables(cons_noR, g_noR, rr, zz, inner)
cons_vec, cons_density         = change_of_variables(cons    , g    , rr, zz, inner)
cons_wc_vec, cons_wc_density   = change_of_variables(cons    , g_wc , rr, zz, inner)

# Consumption dynamics
logC_mu_noR, logC_sigma_noR = consumption_dynamics(cons_noR, rr, zz,
                                                    mu_1_noR, mu_r_noR, mu_z_noR,
                                                    model, inner);
logC_mu, logC_sigma         = consumption_dynamics(cons, rr, zz,
                                                    mu_1, mu_r, mu_z,
                                                    model, inner);
logC_mu_wc, logC_sigma_wc   = consumption_dynamics(cons, rr, zz,
                                                    mu_1_wc, mu_r_wc, mu_z_wc,
                                                    model, inner);


logK12_sigma = sqrt.((sigma_k1[1]*one_pii + sigma_k2[1]*pii).^2 +
                        (sigma_k1[2]*one_pii + sigma_k2[2]*pii).^2 +
                        (sigma_k1[3]*one_pii + sigma_k2[3]*pii).^2)[inI, inJ];
logK12_sigma *= 0.01
drdz = dr*dz


riskfree = zeros(II, JJ);
rf = risk_free_rate(cons, rr, zz, mu_1_wc, mu_r_wc, mu_z_wc, delta, model, inner);
riskfree[inI, inJ] .= rf
riskfree[inner+1:end-inner, 1:inner] .= rf[:, 1]
riskfree[inner+1:end-inner, end-inner:end] .= rf[:, end]
riskfree[1:inner, :] .= riskfree[inner+1:2*inner, :]
riskfree[end-inner+1:end, :] .= riskfree[end-2*inner:end-inner-1, :]

rf_vec, rf_density   = change_of_variables(riskfree, g, rr, zz, inner)

println(" (5) Calibration targets                                     ")
println("    dlogC local mean: ", sum(g[inI,inJ] .* logC_mu * drdz));
println("    dlogC local vol : ", sum(g[inI,inJ] .* logC_sigma * drdz));
println("    dlogK local vol : ", sum(g[inI,inJ] .* logK12_sigma * drdz));
println("    C/I ratio:        ", sum(g .* cons ./ (one_pii .* d1 + pii .* d2))*drdz);
println("    I/K ratio:        ", sum(g .* (one_pii .* d1 + pii .* d2)) * drdz);
println("    dlogC_wc mean   : ", sum(g[inI,inJ] .* logC_mu_wc * drdz));
println("    riskfree rate   : ", sum(g[inI,inJ] .* rf * drdz));
println("=============================================================")


### Calculate Impulse Response Functinos

In [ ]:
#==============================================================================#
# Impulse Response Functions and Term structure of uncertainty prices
#==============================================================================#

# Which shock and how big
dW0 = Matrix(1.0 * I, 3, 3);

# Deciles under the baseline stationary distribution with robust decisions
gz_cdf = vec(cumsum(sum(g, dims=1)'* dz*dr, dims=1))
gr_cdf = vec(cumsum(sum(g, dims=2) * dz*dr, dims=1))
ind_r1dec = findfirst(x -> x >= 0.1, gr_cdf)
ind_r5dec = findfirst(x -> x >= 0.5, gr_cdf)
ind_r9dec = findfirst(x -> x >= 0.9, gr_cdf)

ind_z1dec = findfirst(x -> x >= 0.1, gz_cdf)
ind_z5dec = findfirst(x -> x >= 0.5, gz_cdf)
ind_z9dec = findfirst(x -> x >= 0.9, gz_cdf)

start_p = [[ind_r5dec, ind_z5dec],
            [ind_r1dec, ind_z5dec],
            [ind_r9dec, ind_z5dec],
            [ind_r5dec, ind_z1dec],
            [ind_r5dec, ind_z9dec]]

# horizon
hor = 1000
N = size(start_p)[1]                    # number of initial states

pii_irf = zeros(hor, 3, 2, N);
z_irf = zeros(hor, 3, 2, N);
price_12 = zeros(N, hor);
price_z = zeros(N, hor);


if compute_irfs == 1

    println(" (6) Compute Impulse Resonse Functions (hor=", hor, ")       ")
    d1_interp = LinearInterpolation((rr[:, 1], zz[1, :]), d1);
    d2_interp = LinearInterpolation((rr[:, 1], zz[1, :]), d2);
    P = factorize(sparse(1I, II*JJ, II*JJ) - A');
    P_wc = factorize(sparse(1I, II*JJ, II*JJ) - A_wc');
    P_noR = factorize(sparse(1I, II*JJ, II*JJ) - A_noR');

    hor_p = Int64[hor for i=1:N]
    dW0_p = Matrix{Float64}[dW0 for i=1:N]
    P_p = SuiteSparse.UMFPACK.UmfpackLU{Float64,Int64}[P for i=1:N]
    P_wc_p = SuiteSparse.UMFPACK.UmfpackLU{Float64,Int64}[P_wc for i=1:N]
    P_noR_p = SuiteSparse.UMFPACK.UmfpackLU{Float64,Int64}[P_noR for i=1:N]
    pii_p = Matrix{Float64}[pii for i=1:N]
    zz_p = Matrix{Float64}[zz for i=1:N]
    d1_interp_p = interpol[d1_interp for i=1:N]
    d2_interp_p = interpol[d2_interp for i=1:N]
    model_p = TwoCapitalEconomy[model for i=1:N]
    policies_p = PolicyFunctions[policies for i=1:N]
    grid_p = Grid_rz[grid for i=1:N]

    label_p = [1, 2, 3, 4, 5]

    res = pmap(IRF, start_p, dW0_p, hor_p, P_p, P_wc_p, P_noR_p,
                    pii_p, zz_p, d1_interp_p, d2_interp_p,
                    model_p, policies_p, grid_p, label_p)

    label_vec = [res[i][5] for i=1:N]

    for (i, spec) in enumerate(label_vec)
        pii_irf[:, :, :, spec] = res[i][1]
        z_irf[:, :, :, spec] = res[i][2]
        price_12[spec, :] = res[i][3]
        price_z[spec, :] = res[i][4]
    end
    println("=============================================================")
end


### Save results

In [ ]:
results = Dict("delta" => delta,
# Single capital
"alpha_c_hat" => alpha_c_hat, "beta_hat" => beta_hat,
"alpha_z_hat" => alpha_z_hat, "kappa_hat" => kappa_hat,
"sigma_c" => sigma_c, "sigma_z_1cap" => sigma_z_1cap,
"zbar" => zbar, "cons_1cap" => cons_1cap, "stdev_z_1cap" => stdev_z_1cap,
"H0" => H0, "H1" => H1,
# Two capital stocks
"alpha_k1_hat" => alpha_k1_hat, "alpha_k2_hat" => alpha_k2_hat,
"beta1_hat" => beta1_hat, "beta2_hat" => beta2_hat,
"sigma_k1" => sigma_k1, "sigma_k2" => sigma_k2,
"sigma_z" =>  sigma_z, "A1" => A1, "A2" => A2, "phi1" => phi1, "phi2" => phi2,
"alpha_z_tilde" => alpha_z_tilde, "kappa_tilde" => kappa_tilde,
"alpha_k1_tilde" => alpha_k1_tilde, "beta1_tilde" => beta1_tilde,
"alpha_k2_tilde" => alpha_k2_tilde, "beta2_tilde" => beta2_tilde,
"xi0" => xi0, "xi1" => xi1, "xi2" => xi2,
"I" => II, "J" => JJ,
"rmax" => rmax, "rmin" => rmin, "zmax" => zmax, "zmin" => zmin,
"rr" => rr, "zz" => zz, "pii" => pii, "dr" => dr, "dz" => dz, "T" => hor,
"maxit" => maxit, "crit" => crit, "Delta" => Delta, "inner" => inner,
# Without robustness
"V_noR" => V_noR, "val_noR" => val_noR,
"d1_F_noR" => d1_F_noR, "d2_F_noR" => d2_F_noR,
"d1_B_noR" => d1_B_noR, "d2_B_noR" => d2_B_noR,
"d1_noR" => d1_noR, "d2_noR" => d2_noR,
"g_noR_dist" => g_noR_dist, "g_noR" => g_noR,
"mu_1_noR" => mu_1_noR, "mu_r_noR" => mu_r_noR, "mu_z_noR" => mu_z_noR,
# Robust control under baseline
"V0" => V0, "V" => V, "val" => val, "ell_star" => ell_star,
"d1_F" => d1_F, "d2_F" => d2_F,
"d1_B" => d1_B, "d2_B" => d2_B,
"d1" => d1, "d2" => d2,
"h1_F" => policies.h1_F, "h2_F" => policies.h2_F, "hz_F" => policies.hz_F,
"h1_B" => policies.h1_B, "h2_B" => policies.h2_B, "hz_B" => policies.hz_B,
"h1_dist" => h1_dist, "h2_dist" => h2_dist, "hz_dist" => hz_dist,
"h1" => h1, "h2" => h2, "hz" => hz,
"g_dist" => g_dist, "g" => g,
"mu_1" => mu_1, "mu_r" => mu_r, "mu_z" => mu_z,
# Robust control under worst-case
"g_wc_dist" => g_wc_dist, "g_wc" => g_wc,
"mu_1_wc" => mu_1_wc, "mu_r_wc" => mu_r_wc, "mu_z_wc" => mu_z_wc,
# Non-robust control under worst-case
"g_wc_noR_dist" => g_wc_noR_dist, "g_wc_noR" => g_wc_noR,
# Distortion measures
"re" => re, "q" => q,
"chernoff" => chernoff, "halflife" => halflife,
# Local uncertainty prices (stationary distributions)
"h12_vec" => h12_vec, "h12_density" => h12_density,
"hz_vec" => hz_vec, "hz_density" => hz_density,
# Risk-free rate (stationary distributions)
"riskfree" => riskfree,
"rf_vec" => rf_vec, "rf_density" => rf_density,
# Consumption (stationary distributions)
"cons_noR" => cons_noR, "cons" => cons,
"cons_noR_vec" => cons_noR_vec, "cons_noR_density" => cons_noR_density,
"cons_vec" => cons_vec, "cons_density" => cons_density,
"cons_wc_vec" => cons_wc_vec, "cons_wc_density" => cons_wc_density,
# Consumption (drift and volatilities)
"logC_mu_noR" => logC_mu_noR, "logC_sigma_noR" => logC_sigma_noR,
"logC_mu" => logC_mu, "logC_sigma" => logC_sigma,
"logC_mu_wc" => logC_mu_wc, "logC_sigma_wc" => logC_sigma_wc,
# Impulse Response Functions
"R_irf" => pii_irf, "Z_irf" => z_irf,
# Expected future uncertainty prices
"shock_price_12" => price_12, "shock_price_z" => price_z,
# Calibration
"A_1cap" => A_1cap, "phi_1cap" => phi_1cap, "alpha_k_hat" => alpha_k_hat,
"consumption_investment" => consumption_investment, "investment_capital" => investment_capital)

npzwrite("./data/" * filename, results)